# Predictive Brain Tumor Image AI Project - Data Wrangling

To make an image-processing brain tumor predictive model to automate on scale.

# Initial Setup

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt